# 3.2 - Validações holdout

## 3.2.1 - Introdução:

Antes de definir o método holdout, precisamos entender o que é validação cruzada:
* **Validação Cruzada (Cross Validation):** É uma técnica que avalia a capacidade de generalização de um modelo. É muito usada quando fazemos a predição de valores. Consiste na partição dos dados em dados de treino e dados de teste, essa divisão é feita em uma porcentagem.
<img src="https://miro.medium.com/max/1056/1*2fW8Y8gDEgUMIWO8z3-cLQ.png" width=500px></img>


* **Holdout:** É um tipo simples de validação cruzada. Este método consiste na divisão dos dados em duas partes, uma parte para **treino** e a outra parte para **teste**, onde antes dos dados serem testados, eles precisam passar pelo treinamento para que seja feito previsões do modelo treinado sobre o restante dos dados. Esse método faz uma divisão aleatória de 70% dos dados para treino e 30% para teste. A imagem abaixo ilustra esta divisão sobre o total de dados:

<img src="https://static.wixstatic.com/media/ec9f84_2afbd552b2954e31913e725431d7f772~mv2.png/v1/fill/w_515,h_315,al_c/ec9f84_2afbd552b2954e31913e725431d7f772~mv2.png"></img>

O código abaixo exemplifica o uso do método holdout para a divisão dos dados em 70-30:

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
iris.target

# Método Holdout: 70-30
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)
clf = svm.SVC(gamma='auto')

clf.fit(X_train, y_train)
scores = cross_val_score(clf, iris.data, iris.target, cv=5, scoring='accuracy')
print('Mean: ', scores.mean())

Mean:  0.9800000000000001


## 3.2.2 - Leave One Out:

Este método é uma ramificação do K-Fold, onde temos o K igual a N, que define o número total de dados. Outra diferença do K-Fold, é que a estimativa de erro é calculada N vezes, uma para cada dado. Este processo é semelhante método estatístico Jack-Knife. A imagem abaixo ilustra cada uma das iterações de acordo com o total de dados (N):

<img src="https://dataaspirant.com/wp-content/uploads/2020/12/7-LOOCV-Leave-One-Out-Cross-Validation.png" width=500px></img>

O código abaixo exemplifica o uso de Leave One Out em um modelo de classificação Random Forest (a métrica usada para avaliar o modelo foi a acurácia):

In [5]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X, y = make_blobs(n_samples=100, random_state=1)
cv = LeaveOneOut()

y_true, y_pred = list(), list()
for train_ix, test_ix in cv.split(X):
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]

    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, y_train)

    yhat = model.predict(X_test)

    y_true.append(y_test[0])
    y_pred.append(yhat[0])

acc = accuracy_score(y_true, y_pred)
print('Accuracy: %.3f' % acc)

Accuracy: 0.990


## 3.2.3 - K-Fold:

É uma técnica simples de validação cruzada. É conhecida como K-Fold, pois o K significa o número de subdivisões feitas, e o Fold significa cada um dos blocos de cada k, ou seja, esta técnica consiste em uma divisão dos dados em k subpartes, após isso o método holdout é repetido k vezes. A estimativa de erro é calculada de acordo com as k tentativas para obter a eficácia do modelo. A imagem abaixo representa o uso do K-Fold, onde k = 5:

<img src="https://miro.medium.com/max/601/0*O_491U1UfF1lIqz_.png"></img>

O código abaixo exemplifica o uso de K-Fold em um modelo de Regressão Logística (a métrica usada para avaliar o modelo foi a acurácia):

In [3]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LogisticRegression()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.868 (0.032)


## 3.2.4 - Out of Sample:

Existem os dados amostrais que usamos e os que ficam de fora da amostra. Esses dados que nós usamos são chamados de In-sample, pois os modelos são aplicados com base nesses dados. E os dados que ficaram de fora são chamados de Out-of-sample, pois eles são "ignorados" pelo modelo.

Quando estamos tentando aplicar os modelos no mercado financeiro, podemos dizer que estamos buscando pelos dados que estão fora da amostra, pois o modelo vai prever por exemplo, os valores futuros de fechamento dos candles com base em dados históricos de um ativo.

A imagem abaixo ilustra os dados In-sample e Out-of-sample:


<img src="https://i.stack.imgur.com/6cviv.png" width=500px></img>

## 3.2.5 - Out of Time:

A validação Out-of-time é definida como a tentativa de testar os dados em um período que está fora do previsto, ou seja, se o modelo foi treinado desde 2017-2019 com base em dados históricos, e se o modelo for testado, mas agora no ano de 2020, estamos fazendo uma validação Out-of-time. A vantagem de se usar essa abordagem é o que modelo está abilitado a identificar valores fora do intervalo que era previsto. Porém isso não é levado em consideração na hora de construir o modelo, pois selecionamos um modelo que tenha um bom desempenho nos dados de validação In-time. A imagem abaixo ilustra essa abordagem:

<img src="https://www.researchgate.net/profile/Nathaniel-Bastian/publication/330870705/figure/fig4/AS:722795965194240@1549339395552/Spatial-Out-of-time-validation.ppm" width=700px></img>

## 3.2.6 - Conclusão:

Usamos os métodos de validação no mercado financeiro, através da análise de dados, pois quando importamos os dados, dividimos os dados para treino e para teste. Um método usado é o Out-of-sample, que abilita o modelo a ser usado em dados que ainda não foram vistos.

<img src="https://aostrading.cz/wp-content/uploads/2015/08/In-Sample-and-Out-of-Sample-test.jpg"></img>